## Automate Event Planning

Maxim's remarks

In [2]:
#!pip install -U crewai crewai_tools langchain_community

In [3]:
from crewai import Agent, Task, Crew
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

default_llm = AzureChatOpenAI(
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_KEY")
)

# from langchain_core.messages import HumanMessage
# message = HumanMessage(
#     content="Translate this sentence from English to French. I love programming."
# )
# r = default_llm.invoke([message])
# print(r)

## Tools

In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md') # RAG over Markdown file

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]


## Agents

In [5]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    llm = default_llm,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do incredible research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm = default_llm,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm = default_llm,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm = default_llm,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Tasks

In [6]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

# You can pass a list of tasks as context to a task.
# The task then takes into account the output of those tasks in its execution.
# The task will not run until it has the output(s) from those tasks.

# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

## Crew

In [7]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

In [8]:
# Example data for kicking off the process
job_application_inputs = {
    'job_posting_url': 'https://www.metacareers.com/jobs/1521471421750740/',
    'github_url': 'https://github.com/maxim-saplin',
    'personal_writeup': """NONE"""
}

### this execution will take some time to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://www.metacareers.com/jobs/1521471421750740/) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/maxim-saplin) URLs, and personal write-up (NONE). Utilize tools to extract and synthesize information from these sources.
 [DEBUG]: == [Personal Profiler for Engineers] Task output: 


 [DEBUG]: == Working Agent: Resume Strategist for Engineers
 [INFO]: == Starting Task: Using the profile and job requirements obtained from previous tasks, tailor the resume to highlight the most relevant areas. Employ tools to adjust and enhance the resume content. Make sure th

In [9]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Maxim Saplin
- GitHub: https://github.com/maxim-saplin

## Profile
Maxim Saplin is a seasoned Technology Leader, Portfolio Manager, and Open-source Maintainer with over 15 years of experience in software development. His leadership has driven innovation in AI-powered monetization solutions, particularly in the realm of generative AI and dynamic ad creation. He has a proven track record in building software end-to-end, agile project and product development, and portfolio management. His expertise includes generative AI, LLM, ML, and data mining engagements. Maxim is SAFe, Kanban, and SCRUM certified and holds degrees from three universities in computer science, languages, and an MBA. 

Maxim has supervised delivery at scale, managing teams of over 1200 people across the globe, and has been involved in pre-sales and RFx activities, closing seven-figure deals. He has overseen AI strategy development and implementation, ensuring optimization of digital monetization efforts. Maxim has led teams of applied scientists and engineers, managing projects from prototyping to integration, with a strong focus on Computer Vision and Machine Learning technologies. His technical acumen has been pivotal in maturing and integrating advanced systems, collaborating cross-functionally to advance product pipelines across hardware, software, integration, infrastructure, and applications. 

With extensive experience in managing global, multi-disciplinary teams, Maxim excels in recruiting and nurturing world-class talent, establishing metrics, and driving engineering and operational excellence. His interpersonal communication skills enable him to effectively engage with researchers, engineers, and various stakeholders, ensuring alignment and successful project execution. Maxim's leadership style fosters a positive work environment, emphasizing accountability and collaboration. His strategic and tactical approach has been instrumental in setting long-term strategies and roadmaps, leading to the successful productization of AI and Machine Learning technologies. His rare fusion of technical expertise and people leadership makes him an ideal candidate for the Engineering Manager, GenAI Monetization role at Meta.

## Work History

### EPAM Systems: Delivery Partner, Product/Program Management (Jul 2021 - Present)
- Off-shore Delivery Partner for the Northern California business unit, supervising delivery at a scale of 1000+ people.
- Responsible for a large portfolio of software projects executed for clients from Silicon Valley, ensuring "Green" delivery with capacity to staff key positions and transform delivery processes.
- Acted as an escalation point for portfolio issues, integrated HR and People Management, liaised with RTB teams, and participated in tech pre-sales.
- Launched new engagements, scaled delivery locations, and contributed to over 50% business growth in the unit.

### EPAM Systems: GDO/DMO Advisor (Sep 2022 - Present)
- Part of the Global Delivery Office (GDO) and Delivery Management Office (DMO), tasked with instituting best practices and pursuing delivery excellence.
- Prototyped and piloted Large Language Models integration into in-house Delivery Metrics products.
- Led developer productivity and Git analytics research, pivoting to skill extraction and building developer profiles.
- Scaled Delivery Partner role and piloted People Health Check Framework.

### EPAM Systems: Program Manager (Jan 2022 - Apr 2022)
- Joined at pre-sales stage and converted a Big Tech lead into a running program with multiple streams.
- Successfully transferred the program to a dedicated manager and zoomed out to portfolio management.

### ScienceSoft: Head of Microsoft Department (Sep 2013 - Apr 2021)
- Led the Microsoft Tech direction of the company with 60+ engineers.
- Involved in pre-sales/RFPs, starting new engagements, and supervising ongoing projects.
- Managed multiple teams with 100+ people, working on a wide range of technologies and solutions.

### ScienceSoft: Project/Program Manager (Feb 2012 - Aug 2013)
- Managed multiple projects for French/UK customers, including .NET/Web development and BI solutions.
- Supervised a large group of people across different locations and integrated multiple software products into a single ecosystem.

### ScienceSoft: Senior .NET Developer/Team Lead (Sep 2009 - Feb 2012)
- Started as the first developer on a .NET project and grew into overseeing technical implementation and team management.
- Involved in estimates and interviews, and took part in smaller projects and iPad app development.

### Intetics: .NET Developer (Apr 2009 - Aug 2009)
- Worked as an ASP.NET developer on a US-based e-commerce product.

### Omertex: Junior .NET Developer (Apr 2008 - Mar 2009)
- Joined a start-up company and participated in multiple ASP.NET web projects.

## Education

### MBA in Business Statistics, Machine Learning, and Data Mining
School of Business, Belarusian State University (2012-2014)

### Linguistics
Belarusian State Academy of Post-diploma Education (2006-2009)

### M.Sc. in Computer Science
Belarusian National Technical University (2004-2010)

## Certifications

### Scaled Agile
- SAFe® certifications version 6 (2023)
- Certified SAFe® 5 Lean Portfolio Manager, Scrum Master, DevOps Practitioner, Product Owner/Product Manager, Agilist (2022)

### Kanban University
- Kanban Management Professional, Kanban Systems Improvement, Kanban System Design (2022)

### Scrum.org
- Professional Scrum Master™ I (PSM I) (2018)

### Other Certifications
- International Business (Olivier P. Roche, Salisbury University) (2016)
- Silicon Valley Business Culture (Andrew F. Wood, San Jose State University) (2015)
- Model Thinking, University of Michigan, Coursera (2012)

## Open Source Projects
- cptX: “Headless” AI Coding Assistant for VSCode
- data_table_2: Flutter widget library, top 1% by popularity in Dart’s package manager Pub
- CPDT: Cross-platform disk benchmark with over 500k+ downloads at Google Play alone
- Puma.NET: First free-to-use OCR library for .NET in 2009

## Online Profiles
- LinkedIn: [Maxim Saplin](https://www.linkedin.com/in/maxim-saplin/)
- GitHub: [Maxim Saplin](https://github.com/maxim-saplin)
- Stack Overflow: [Maxim Saplin](https://stackoverflow.com/users/440696/maxim-saplin)
- Dev.to: [Maxim Saplin](https://dev.to/maximsaplin)
- WeAreCommunity: [Maxim Saplin](https://wearecommunity.io/users/maxim-saplin)

In [10]:
display(Markdown("./interview_materials.md"))

### Potential Interview Questions and Talking Points for Maxim Saplin

#### **General Questions:**
1. **Can you tell us about your background and how your experience aligns with the role of Engineering Manager for GenAI Monetization at Meta?**
   - *Talking Point:* Highlight over 15 years of experience in software development, leadership in AI-powered monetization solutions, and managing large teams globally.
   
2. **What interests you about the GenAI Monetization role at Meta?**
   - *Talking Point:* Express passion for AI and digital monetization, enthusiasm for Meta's innovative culture, and desire to contribute to cutting-edge AI technologies.

#### **Technical Leadership and Management:**
3. **Can you describe a time when you led a multi-disciplinary team on a complex AI/ML project?**
   - *Talking Point:* Mention leading applied scientists and engineers, managing projects from prototyping to integration, and ensuring alignment across hardware, software, and infrastructure.

4. **How do you approach recruiting and managing a world-class technical team?**
   - *Talking Point:* Discuss experience in recruiting top talent, performance management, and creating a positive work environment based on accountability and collaboration.

5. **What strategies do you use to ensure engineering and operational excellence within your team?**
   - *Talking Point:* Highlight establishing metrics and processes for regular assessment and improvement, fostering continuous improvement and best practices.

#### **AI and Machine Learning:**
6. **Can you elaborate on your experience with generative AI and its application in ad creation or digital monetization?**
   - *Talking Point:* Share insights from projects like Microsoft Autogen and cptX, emphasizing the development of AI-powered solutions for dynamic and personalized ad creatives.

7. **How do you stay current with advancements in AI/ML technologies, and how do you integrate these into your projects?**
   - *Talking Point:* Mention continuous learning, participation in tech communities, and integrating cutting-edge technologies like LLMs and machine vision into projects.

#### **Project Management and Cross-functional Collaboration:**
8. **Describe a challenging project where you had to coordinate with various cross-functional teams. How did you ensure successful execution?**
   - *Talking Point:* Discuss leading large-scale projects involving cross-functional teams, aligning goals, and maintaining clear communication to drive project success.

9. **How do you balance strategic and tactical responsibilities in your role?**
   - *Talking Point:* Explain the approach to setting long-term strategies and roadmaps while staying engaged with day-to-day development activities.

#### **Specific to Meta and GenAI:**
10. **What do you see as the future direction of GenAI in digital monetization, and how would you contribute to this vision at Meta?**
    - *Talking Point:* Share thoughts on the potential of GenAI in transforming digital monetization, and outline how your experience and skills can drive innovation and growth in this area.

11. **How would you handle the integration of advanced AI systems with existing product architectures at Meta?**
    - *Talking Point:* Discuss your experience in maturing and integrating advanced systems, collaborating with respective teams to create robust product architectures.

#### **Behavioral Questions:**
12. **Can you give an example of a time when you had to manage a significant conflict within your team? How did you resolve it?**
    - *Talking Point:* Provide a specific example, focusing on conflict resolution skills, communication, and maintaining team morale.

13. **Describe a situation where you had to adapt to a major change or pivot in a project. How did you manage it?**
    - *Talking Point:* Highlight resilience and adaptability, discussing how you led your team through changes and ensured project continuity.

#### **Personal Development and Continuous Improvement:**
14. **How do you encourage and support continuous learning and development within your team?**
    - *Talking Point:* Emphasize promoting self-education, providing resources for professional development, and fostering a culture of continuous improvement.

15. **What are some of your proudest achievements in your career so far, and how do they demonstrate your fit for this role?**
    - *Talking Point:* Highlight key achievements such as leading successful AI projects, scaling delivery teams, and contributing to significant business growth.

By preparing responses to these questions and focusing on the key talking points, Maxim Saplin can effectively highlight his qualifications, experience, and suitability for the Engineering Manager, GenAI Monetization position at Meta.

In [11]:
print(job_application_crew.usage_metrics)

{'total_tokens': 93093, 'prompt_tokens': 82274, 'completion_tokens': 10819, 'successful_requests': 26}
